In [ ]:
#default_exp prepare_dataset

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# export
from IPython.display import display,HTML
from typing import Any, Callable, Optional, List, Dict, Iterator, Set, Tuple
import shutil ###############

In [ ]:
# export
from fastai.vision.all import *
import PIL
import torch
import numpy as np
import Imath, OpenEXR

In [ ]:
# export
from fastprogress.fastprogress import master_bar, progress_bar

from upscalers_pytorch.config import data_folder, DatasetSpec
from upscalers_pytorch.image import load_exr, save_exr, show_img
from upscalers_pytorch.utils import *

# Overview

> `prepare_dataset`: functions to show, read, write, decimate in common formats, e.g. PNG and EXR.

# Initialize saving folder

In [ ]:
test_folder = TestFolder("prepare_dataset")

# Data manipulation

## `save_downscaled_exr_decimate`

In [ ]:
# export
#this functions is similar to save_downscaled_exr in 0030_image (move it to another notebook)
import cv2
def save_downscaled_exr_decimate(in_path:Path, out_path:Path, width:int, height:int, frame_index:int, show=False):
    """Save a downscaled copy of the EXR image from `in_path`. Save the `width`x`height` result in `out_path`.
    
        Assumptions:
            - Input, output are 16-bit EXR
            - Width/height aspect ratio is preserved    """
    
    img_hr = load_exr(str(in_path))
    KernelSize = (img_hr.shape[2]//width, img_hr.shape[1]//height)
    img_lr = image_decimate(img_hr, KernelSize, frame_index) # low-res image
    save_exr(img_lr, str(out_path))
    
    if show: show_img(img_lr, figsize=(15,8))

# Data Decimation Routines

## pixel_to_img_coords_with_shape

In [ ]:
# export
#this functions is similar to pixel_to_img_coords in 0180_TAA (move it to another notebook)
def pixel_to_img_coords_with_shape(xy_coords, img_width:int, img_height:int, xy_offset=None):
    """Convert xy coordinates expressed in pixels (e.g. from motion vectors) into a range of [-1,1].
    
        Args:
            xy_coords: (N,H,W,2) where the last axis is an absolute (x,y) coordinate expressed in pixels.
            img_width: image width
            img_height: image height
        
        Return:
            xy_coords: (N,H,W,2) where the last axis should range between [-1,1], except if the coordinates were out-of-image."""
    if xy_offset is None:
        xy_offset = tensor([0.0, 0.0])
    
    # TODO: think whether this should be detached...? do we need to propagate gradients?
    xy_coords = xy_coords.clone().detach()
    
    xy_coords[..., 0:1] = (xy_coords[..., 0:1] + xy_offset[0]) / (img_width-1) * 2 - 1.0 # x coordinates
    xy_coords[..., 1:2] = (xy_coords[..., 1:2] + xy_offset[1]) / (img_height-1) * 2 - 1.0 # y coordinates
    return xy_coords

## get_grid

In [ ]:
# export
def get_grid(shapeHr, KernelSize, anchor):
    """Creates a grid for xy coordinates of pixels into range of [-1,1]
    
        Args:
            shapeHr: shape of given image (High resolution image)
            KernelSize: size of kernel in (x,y) coordinates manner
            anchor: position of pixel around of center of the kernel size
            
        Return:
            xy_coords: (N,H,W,2) where the last axis should range between [-1,1], except if the coordinates were out-of-image.
    """
    x = torch.arange(start = 0, end = shapeHr[-1], dtype = torch.float32)
    y = torch.arange(start = 0, end = shapeHr[-2], dtype = torch.float32)
    #create xy offsets
    x_offset = x[KernelSize[0] // 2 - 1 + anchor[0]::KernelSize[0]]
    y_offset = y[KernelSize[1] // 2 - 1 + anchor[1]::KernelSize[1]]
    #mesh grid for pixel indexes of x and y coordinates
    meshy, meshx = torch.meshgrid((y_offset, x_offset))
    #created mesh
    stacked_mesh = torch.stack([meshx, meshy], axis=-1).unsqueeze(0)
    
    #coordinates grid
    return pixel_to_img_coords_with_shape(stacked_mesh, shapeHr[-1], shapeHr[-2])

## halton

In [ ]:
# export
def halton(index, base):
    """Creates a halton index value
    
        Args:
            index: number/index of sequence
            base: base of given sequence
            
        Return:
            result: value of given sequence
    """
    f = 1.0
    result = 0.0
    i = index
    while i > 0:
        f /= base
        result += f * (i % base)
        i = i // base
        




        
    return result 

## halton_sequence

In [ ]:
#export
def halton_sequence(frame_index, KernelSize):
    """Creates a halton sequence
    
        Args:
            frame_index: index of frame in given video
            KernelSize: size of kernel in (x,y) coordinates manner
            
        Return:
            (x,y): returns a value of pixel coordinate
    """
    jitter_index = 1 + (frame_index & 0xf)

    jitter_x = (2 * halton(jitter_index, 2)) - 1
    jitter_y = (2 * halton(jitter_index, 3)) - 1

    x = int(KernelSize[0] // 2 * jitter_x + 0.5)
    y = int(KernelSize[1] // 2 * jitter_y + 0.5)
    
    return (x,y)

In [ ]:
# halton sequence algorithm for testing (found at the internet)
from math import log, floor, ceil, fmod

def halton_other(dim, nbpts):
    h = np.empty(nbpts * dim)
    h.fill(np.nan)
    p = np.empty(nbpts)
    p.fill(np.nan)
    P = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31]
    lognbpts = log(nbpts + 1)
    for i in range(dim):
        b = P[i]
        n = int(ceil(lognbpts / log(b)))
        for t in range(n):
            p[t] = pow(b, -(t + 1) )

        for j in range(nbpts):
            d = j + 1
            sum_ = fmod(d, b) * p[0]
            for t in range(1, n):
                d = floor(d / b)
                sum_ += fmod(d, b) * p[t]

            h[j*dim + i] = sum_

    return h.reshape(nbpts, dim)

In [ ]:
# just for testing
def get_halton_sequence(length:int, KernelSize):
    """
        Functions generates a np.zeros of halton's sequence
    """
    sequence = np.zeros((length,2))

    for i in range(length):
        sequence[i] = halton_sequence(i, KernelSize)
    return sequence

In [ ]:
# TODO: scatter plot (ideally with numbers)
# Test for 8,8 grid with length of 8 halton sequence
length = 16
KernelSize = (8,8)

t_sequence = get_halton_sequence(length, KernelSize)
t_other_sequence = halton_other(2, length)
# print(sequence)
# print(other_sequence)


In [ ]:
fig = plt.figure(figsize=(15,15))
plt.plot(t_sequence[:length, 0], t_sequence[:length, 1], 'o', color='red', label='Position of sequence point')
plt.title("uniform-distribution halton ({0})".format(length))
plt.legend()
for counter, (x, y) in enumerate(zip(t_sequence[:,0], t_sequence[:, 1])):
    plt.text(x, y, str(counter + 1), color="black", fontsize=12)

In [ ]:
plt.clf()

In [ ]:
fig = plt.figure(figsize=(15,15))
plt.plot(t_other_sequence[:length, 0], t_other_sequence[:length, 1], 'x', color='green', label='Position of sequence point')
plt.title("uniform-distribution other halton ({0})".format(length))
plt.legend()
for counter, (x, y) in enumerate(zip(t_other_sequence[:,0], t_other_sequence[:, 1])):
    plt.text(x, y, str(counter + 1), color="black", fontsize=12)

In [ ]:
plt.clf()

In [ ]:
#Tile checking

length = 16
KernelSize = (8,8)

t_sequence = get_halton_sequence(length, KernelSize)
#t_sequence = halton_other(2,length)
# t_sequence

In [ ]:
#for testing
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

In [ ]:
points_amount = 8 #how many points should be plotted out
cmap = get_cmap(points_amount + 1)

In [ ]:
# For 8 points
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(20,20))
ax = axes.ravel()

plt.subplots_adjust(left=2, bottom=2, right=2.5, top=2.5, wspace=0.001, hspace=0.001) #layout of plot's grid


for i in range(0, ax.shape[0]):
    for j in range(0, points_amount):
        ax[i].plot(t_sequence[j, 0], t_sequence[j, 1], 'o', color=cmap(j))
        
    ax[i].grid()
    
    for counter, (x, y) in enumerate(zip(t_sequence[:points_amount,0], t_sequence[:points_amount, 1])):
        ax[i].text(x + 0.15, y, str(counter + 1), color="black", fontsize=12)

    ax[i].set_xlim([-4, 5])
    ax[i].set_ylim([-4, 5])
    
    ax[i].set_xticks(np.arange(-4, 5, 1))
    ax[i].set_yticks(np.arange(-4, 5, 1))
    ax[i].set_xticklabels([])
    ax[i].set_yticklabels([])

In [ ]:
print(t_sequence[:length, 0]) #x
print(t_sequence[:length, 1]) #y
plt.clf()

In [ ]:
points_amount = 16

In [ ]:
#for 16 points

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(20,20))
ax = axes.ravel()

plt.subplots_adjust(left=2, bottom=2, right=2.5, top=2.5, wspace=0.001, hspace=0.001) #layout of plot's grid


for i in range(0, ax.shape[0]):
    for j in range(0, points_amount):
        ax[i].plot(t_sequence[j, 0], t_sequence[j, 1], 'o', color=cmap(j))
        
    ax[i].grid()
    
    for counter, (x, y) in enumerate(zip(t_sequence[:points_amount,0], t_sequence[:points_amount, 1])):
        ax[i].text(x + 0.15, y, str(counter + 1), color="black", fontsize=12)

    ax[i].set_xlim([-4, 5])
    ax[i].set_ylim([-4, 5])
    
    ax[i].set_xticks(np.arange(-4, 5, 1))
    ax[i].set_yticks(np.arange(-4, 5, 1))
    ax[i].set_xticklabels([])
    ax[i].set_yticklabels([])

In [ ]:
print(t_sequence[:length, 0]) #x
print(t_sequence[:length, 1]) #y
plt.clf()

## data_decimate

In [ ]:
# export
def data_decimate(tensorHr, KernelSize, anchor):
    """Decimates a given image
    
        Args:
            tensorHr: high resolution torch tensor (HR image)
            KernelSize: size of kernel in (x,y) coordinates manner
            anchor: position of pixel around of center of the kernel size
            
        Return:
            TensorImage: decimated tensor
    """
    n = tensorHr.shape[0]
    w = tensorHr.shape[2] // KernelSize[0]
    h = tensorHr.shape[1] // KernelSize[1]
    grid = get_grid(tensorHr.shape, KernelSize, anchor)
    tensor = F.grid_sample(tensorHr.unsqueeze(0).float(), grid, mode='nearest') 
    
    return TensorImage(tensor.squeeze(0))

## image_decimate

In [ ]:
# export
def image_decimate(tensorHr, KernelSize, frame_index):   
    """Decimates an image at +given video frame
    
        Args:
            tensorHr: high resolution torch tensor (HR image)
            KernelSize: size of kernel in (x,y) coordinates manner
            frame_index: frame index in given video
            
        Return:
             data_decimate: decimated photo from HR to LR
    """
    anchor = halton_sequence(frame_index, KernelSize)
    
    return data_decimate(tensorHr, KernelSize, anchor)


In [ ]:
# just for testing
def data_decimate_xyloop(tensorHr, KernelSize, anchor):
    n = tensorHr.shape[0]
    w = tensorHr.shape[2] // KernelSize[0]
    h = tensorHr.shape[1] // KernelSize[1]
    offset_x = anchor[0] + (KernelSize[0] // 2) - 1
    offset_y = anchor[1] + (KernelSize[1] // 2) - 1
    
    tensor = torch.zeros((n, h, w), dtype=torch.float32)
                            
    for y in range(0, h):
        for x in range(0, w):
            tensor[:,y,x] = tensorHr[:,y*KernelSize[1]+offset_y,x*KernelSize[0]+offset_x]
   
    return TensorImage(tensor) 


In [ ]:
exr_path = test_folder.get_path("Infiltrator_F88_4K.exr", False)(data_decimate_xyloop)
assert(exr_path.exists())

In [ ]:
exr = OpenEXR.InputFile(str(exr_path))
hr = load_exr(exr_path)
KernelSize = (8,8)
#anchor = (1,-3) # valid for 88th frame
anchor = (1,1)

In [ ]:
lr = data_decimate(hr, KernelSize, anchor) #decimation uses grid sample from pytorch
lr_xyloop = data_decimate_xyloop(hr, KernelSize, anchor) #naive method

In [ ]:
diff = lr - lr_xyloop

hlr2 = image_decimate(hr, KernelSize, 88) #88 frame index

diff2 = hlr2 - lr

In [ ]:
show_img(lr, figsize=(15,8), title='Downsampled image')

In [ ]:
show_img(diff, figsize=(15,8), title="difference between grid_sample and xy loop")

In [ ]:
show_img(diff2, figsize=(15,8), title='Jitter difference between Frames 0 and 88')

In [ ]:
# save_exr(hr,"../test/image/save_exr/hr.exr")
# save_exr(lr,"../test/image/save_exr/lr.exr")
# save_exr(diff,"../test/image/save_exr/diff.exr")
# save_exr(diff2,"../test/image/save_exr/diff2.exr")

## get_frame_index_from_name

In [ ]:
#export
def get_frame_index_from_name(in_file):
    res = [int(s) for s in re.findall('\\d+', in_file)]
    return int(res[0])

# Data Manipulation

## `save_downscaled_exr_decimate`

In [ ]:
# export
#this functions is similar to save_downscaled_exr in 0030_image (move it to another notebook)
import cv2
def save_downscaled_exr_decimate(in_path:Path, out_path:Path, width:int, height:int, frame_index:int, show=False):
    """Save a downscaled copy of the EXR image from `in_path`. Save the `width`x`height` result in `out_path`.
    
        Assumptions:
            - Input, output are 16-bit EXR
            - Width/height aspect ratio is preserved    """
    
    img_hr = load_exr(str(in_path))
    KernelSize = (img_hr.shape[2]//width, img_hr.shape[1]//height)
    img_lr = image_decimate(img_hr, KernelSize, frame_index) # low-res image
    save_exr(img_lr, str(out_path))
    
    if show: show_img(img_lr, figsize=(15,8))

In [ ]:
t_in_path = test_folder.get_path("Infiltrator_F88_4K.exr", False)(get_frame_index_from_name)
assert t_in_path.exists()
t_out_path = test_folder.get_path("Infiltrator_F88_540p.exr", False)(get_frame_index_from_name)
frame_index = get_frame_index_from_name(str(t_in_path.name))
frame_index

In [ ]:
save_downscaled_exr_decimate(t_in_path, t_out_path, 960, 540, frame_index, show=True)

## `downscale_exr_folder`

In [ ]:
# export
#this functions is similar to downscale_exr_folder in 0030_image (move it to another notebook)
def downscale_exr_folder_decimate(in_folder:Path, out_folder:Path, width:int, height:int, show=False, jitter=False):
    """Save a downscaled copy of every .exr image in `in_folder`, into `out_path`, with same filenames.
    
        Assumptions: same as `save_downscaled_exr_decimate`"""
    files = list(in_folder.glob("*.exr"))
    for i in progress_bar(range(len(files))):
        in_file = files[i]
        if jitter:
            frame_idx = get_frame_index_from_name(str(in_file.name))
            save_downscaled_exr_decimate(in_file, out_folder/in_file.name, width, height, frame_index, show=show)
        else:
            save_downscaled_exr_decimate(in_file, out_folder/in_file.name, width, height, 0, show=show)
        

In [ ]:
t_in_folder = test_folder.get_path("", False)(downscale_exr_folder_decimate)

In [ ]:
t_in_exr_paths = list(t_in_folder.glob("*.exr"))
t_in_exr_paths

In [ ]:
t_out_folder =  test_folder.get_path("resized/", False)(downscale_exr_folder_decimate)
t_out_folder.mkdir(exist_ok=True)

In [ ]:
#downscale_exr_folder_decimate(t_in_folder, t_out_folder, 960, 540, show=True)
downscale_exr_folder_decimate(t_in_folder, t_out_folder, 960, 540, show=True, jitter=True)

In [ ]:
#Image decimation tests
downscale_factor = 8 #the image will be downscaled 8 times (width/8, height/8)
downscaled_width = (3840 // downscale_factor) #fixed 4k image width
downscaled_height = (2160 // downscale_factor) #fixed 4k image height

print("Downscaled size: {0}x{1}".format(downscaled_width, downscaled_height))

In [ ]:
# # FROM HERE, I HAVE COMMENTED EVERYTHING, DUE TO FOLDER'S PATH 
# # TODO: CHANGE FILE PATHS TO CORRECT DROPBOX PATHS
# #TAA IMAGES
# #its my own folders, due to no free space in disk drive (if needed, change to your own path)
# TAA_in_folder = Path("C:/Users/DAndrysiak/!REPOS/DATASETS/InfiltratorDemo_2/3840x2160_TAA/color")
# assert TAA_in_folder.exists()

In [ ]:
# # Path.mkdir.("../test/downsample_decimation/InfiltratorDemo_2/3840x2160_TAA/out_color")
# TAA_out_folder = Path("../test/downsample_decimation/InfiltratorDemo_2/3840x2160_TAA/out_color")
# if not TAA_out_folder.exists():
#     TAA_out_folder.mkdir(parents=True, exist_ok=False) #parents=True - for creating a InfiltratorDemo_2/3840x2160_TAA/out_color folders

In [ ]:
# downscale_exr_folder_decimate(TAA_in_folder, TAA_out_folder, downscaled_width, downscaled_height)

In [ ]:
#Diff image case

In [ ]:
# TAA_in_exr_paths = list(TAA_in_folder.glob("*.exr"))
# #taa_in_exr_paths #here it displays a whole folder, folder should contains a images from 00 frame to 29 and 88 frame

In [ ]:
# hr_image_TAA = load_exr(TAA_in_exr_paths[10]) #taa_in_exr_paths[10] -> number of image inside the folder

In [ ]:
# show_img(hr_image_TAA, figsize=(15,8), title='HR')

In [ ]:
# TAA_out_exr_paths = list(TAA_out_folder.glob("*.exr")) #list whole folder and get .exr paths
# #taa_out_exr_paths

In [ ]:
# lr_image_TAA = load_exr(TAA_out_exr_paths[10]) #sample photo

In [ ]:
# show_img(lr_image_TAA, figsize=(15,8), title='LR image')

In [ ]:
# diff_0_88_TAA = load_exr(TAA_in_exr_paths[29]) - load_exr(TAA_in_exr_paths[0]) #diff between 1st and 88th frame

In [ ]:
# show_img(diff_0_88_TAA, figsize=(15,8), title='Diff between frame 1st to 29th and 88')

In [ ]:
########################################################################################################################
########################################################################################################################
########################################################################################################################
########################################################################################################################

In [ ]:
# #no_AA IMAGES
# #its my own folders, due to no free space in disk drive (if needed, change to your own path)
# no_AA_in_folder = Path("C:/Users/DAndrysiak/!REPOS/DATASETS/InfiltratorDemo_2/3840x2160_no_AA/color")
# assert no_AA_in_folder.exists()

In [ ]:
# no_AA_out_folder = Path("../test/downsample_decimation/InfiltratorDemo_2/3840x2160_no_AA/out_color")
# if not no_AA_out_folder.exists():
#     no_AA_out_folder.mkdir(parents=True,exist_ok=True) #parents=True - for creating a InfiltratorDemo_2/3840x2160_TAA/out_color folders

In [ ]:
# downscale_exr_folder_decimate(no_AA_in_folder, no_AA_out_folder, downscaled_width, downscaled_height, jitter=True) 

In [ ]:
#Diff image case

In [ ]:
# no_AA_in_exr_paths = list(no_AA_in_folder.glob("*.exr"))

In [ ]:
# hr_image_no_AA = load_exr(no_AA_in_exr_paths[10]) #no_aa_in_exr_paths[10] -> number of image inside the folder

In [ ]:
# show_img(hr_image_no_AA, figsize=(15,8), title='HR')

In [ ]:
# no_AA_out_exr_paths = list(no_AA_out_folder.glob("*.exr")) #list whole folder and get .exr paths

In [ ]:
# lr_image_no_AA = load_exr(no_AA_in_exr_paths[10]) #sample photo

In [ ]:
# show_img(lr_image_no_AA, figsize=(15,8), title='LR image')

In [ ]:
# diff_0_88_no_AA = load_exr(no_AA_in_exr_paths[29]) - load_exr(no_AA_in_exr_paths[0]) #diff between 1st and 88th frame

In [ ]:
# show_img(diff_0_88_no_AA, figsize=(15,8), title='Diff between frame 1st to 29th and 88')

In [ ]:
# Image Decimation test for 2021-04-09_Infiltrator_preview 

In [ ]:
# # TAA
# #its my own folders, due to no free space in disk drive (if needed, change to your own path)
# issue_TAA_in_folder = Path("C:/Users/DAndrysiak/!REPOS/DATASETS/Infiltrator_sample_frames/3840x2160_TAA/color")
# assert issue_TAA_in_folder.exists()

In [ ]:
# issue_TAA_out_folder = Path("../test/downsample_decimation/Infiltrator_sample_frames/3840x2160_TAA/out_color")
# if not issue_TAA_out_folder.exists():
#     issue_TAA_out_folder.mkdir(parents=True,exist_ok=True) #parents=True - for creating a Infiltrator_sample_frames/3840x2160_TAA/out_color

In [ ]:
# downscale_exr_folder_decimate(issue_TAA_in_folder, issue_TAA_out_folder, downscaled_width, downscaled_height) 

In [ ]:
# # no_AA color
# #its my own folders, due to no free space in disk drive (if needed, change to your own path)
# issue_no_AA_color_in_folder = Path("C:/Users/DAndrysiak/!REPOS/DATASETS/Infiltrator_sample_frames/3840x2160_no_AA/color")
# assert issue_no_AA_color_in_folder.exists()

In [ ]:
# issue_no_AA_color_out_folder = Path("../test/downsample_decimation/Infiltrator_sample_frames/3840x2160_no_AA/out_color")
# if not issue_no_AA_color_out_folder.exists():
#     issue_no_AA_color_out_folder.mkdir(parents=True,exist_ok=True) #parents=True - for creating a Infiltrator_sample_frames/3840x2160_no_AA/out_color folders

In [ ]:
# downscale_exr_folder_decimate(issue_no_AA_color_in_folder, issue_no_AA_color_out_folder, downscaled_width, downscaled_height, jitter=True) 

In [ ]:
# # no_AA depth
# #its my own folders, due to no free space in disk drive (if needed, change to your own path)
# issue_no_AA_depth_in_folder = Path("C:/Users/DAndrysiak/!REPOS/DATASETS/Infiltrator_sample_frames/3840x2160_no_AA/depth")
# assert issue_no_AA_depth_in_folder.exists()

In [ ]:
# issue_no_AA_depth_out_folder = Path("../test/downsample_decimation/Infiltrator_sample_frames/3840x2160_no_AA/out_depth")
# if not issue_no_AA_depth_out_folder.exists():
#     issue_no_AA_depth_out_folder.mkdir(parents=True,exist_ok=True) #parents=True - for creating a Infiltrator_sample_frames/3840x2160_no_AA/out_depth folders

In [ ]:
# downscale_exr_folder_decimate(issue_no_AA_depth_in_folder, issue_no_AA_depth_out_folder, downscaled_width, downscaled_height, jitter=True) 

In [ ]:
# # no_AA mv
# #its my own folders, due to no free space in disk drive (if needed, change to your own path)
# issue_no_AA_mv_in_folder = Path("C:/Users/DAndrysiak/!REPOS/DATASETS/Infiltrator_sample_frames/3840x2160_no_AA/mv")
# assert issue_no_AA_mv_in_folder.exists()

In [ ]:
# issue_no_AA_mv_out_folder = Path("../test/downsample_decimation/Infiltrator_sample_frames/3840x2160_no_AA/out_mv")
# if not issue_no_AA_mv_out_folder.exists():
#     issue_no_AA_mv_out_folder.mkdir(parents=True,exist_ok=True) #parents=True - for creating a Infiltrator_sample_frames/3840x2160_no_AA/out_mv folders

In [ ]:
# downscale_exr_folder_decimate(issue_no_AA_mv_in_folder, issue_no_AA_mv_out_folder, downscaled_width, downscaled_height, jitter=True) 

# Export

In [ ]:
from nbdev.export import *
notebook2script()